In [1]:
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pickle

In [2]:
annot = pd.read_csv('/kaggle/input/smartathon-dataset-zip/dataset/train.csv', nrows=9500)
annot = annot.drop(columns=['name'])

In [3]:
x = pd.DataFrame(columns=annot.columns)
classes = list(annot['class'].unique())
for cl in classes:
    if annot.loc[annot['class']==cl, :].shape[0]>=1000:
        x = x.append(annot.loc[annot['class']==cl, :].sample(n=1000, random_state=42), ignore_index=True)
    else:
        x = x.append(annot.loc[annot['class']==cl, :], ignore_index=True)
annot = x

In [4]:
annot = annot[['image_path', 'xmin', 'ymin', 'xmax', 'ymax', 'class']]
annot.head()

,image_path,xmin,ymin,xmax,ymax,class
0,9295e8d25548246e82421d824c6cdfe4.jpg,-2.0,355.0,163.0,452.0,3.0
1,dc13b32699962e9aa3dc8df689056955.jpg,900.0,277.0,949.0,313.0,3.0
2,694bb9f80b7af62744f97dcb8e4e3584.jpg,612.0,155.0,674.0,228.0,3.0
3,19fb2f72c1c1fecb365d87b5e6e02e31.jpg,539.0,279.0,619.0,338.0,3.0
4,cf45a6b57ca336ec1a670888de9bc7c7.jpg,509.0,224.0,583.0,277.0,3.0


In [5]:
annot['image_path'] = "/kaggle/input/smartathon-dataset-zip/dataset/images/" + annot['image_path']
annot.head()

,image_path,xmin,ymin,xmax,ymax,class
0,/kaggle/input/smartathon-dataset-zip/dataset/i...,-2.0,355.0,163.0,452.0,3.0
1,/kaggle/input/smartathon-dataset-zip/dataset/i...,900.0,277.0,949.0,313.0,3.0
2,/kaggle/input/smartathon-dataset-zip/dataset/i...,612.0,155.0,674.0,228.0,3.0
3,/kaggle/input/smartathon-dataset-zip/dataset/i...,539.0,279.0,619.0,338.0,3.0
4,/kaggle/input/smartathon-dataset-zip/dataset/i...,509.0,224.0,583.0,277.0,3.0


In [6]:
annot['class'] = annot['class'].apply(lambda x: int(x))
annot.head()

,image_path,xmin,ymin,xmax,ymax,class
0,/kaggle/input/smartathon-dataset-zip/dataset/i...,-2.0,355.0,163.0,452.0,3
1,/kaggle/input/smartathon-dataset-zip/dataset/i...,900.0,277.0,949.0,313.0,3
2,/kaggle/input/smartathon-dataset-zip/dataset/i...,612.0,155.0,674.0,228.0,3
3,/kaggle/input/smartathon-dataset-zip/dataset/i...,539.0,279.0,619.0,338.0,3
4,/kaggle/input/smartathon-dataset-zip/dataset/i...,509.0,224.0,583.0,277.0,3


In [7]:
data = []
labels = []
bboxes = []
impaths = []

for i in range(annot.shape[0]):
    img = cv2.imread(annot.iloc[i, 0])
    (h, w) = img.shape[:2]
    annot.iloc[i, 1] = float(annot.iloc[i, 1])/w
    annot.iloc[i, 2] = float(annot.iloc[i, 2])/h
    annot.iloc[i, 3] = float(annot.iloc[i, 3])/w
    annot.iloc[i, 4] = float(annot.iloc[i, 4])/h

    img = load_img(annot.iloc[i, 0], target_size=(224, 224))
    img = img_to_array(img)
    data.append(img)
    labels.append(annot.iloc[i, 5])
    bboxes.append((annot.iloc[i,1], annot.iloc[i,2], annot.iloc[i,3], annot.iloc[i,4]))
    impaths.append(annot.iloc[i,0])

In [8]:
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
impaths = np.array(impaths)

In [9]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [10]:
split = train_test_split(data, labels, bboxes, impaths, test_size=0.20, random_state=42)

In [11]:
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]

In [12]:
vgg = VGG19(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))
vgg.trainable = False

2023-01-14 10:17:19.988778: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 10:17:20.092704: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 10:17:20.093457: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 10:17:20.094939: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

80150528/80134624 [==============================] - 0s 0us/step


In [13]:
flatten = vgg.output
flatten1 = Flatten()(flatten)

In [14]:
bboxHead = Dense(128, activation="relu")(flatten1)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",
	name="bounding_box")(bboxHead)

In [15]:
softmaxHead = Dense(512, activation="relu")(flatten1)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation="softmax",
	name="class_label")(softmaxHead)

In [16]:
model = Model(inputs=vgg.input, outputs=(bboxHead, softmaxHead))

In [17]:
losses = {
	"class_label": "categorical_crossentropy",
	"bounding_box": "mean_squared_error",
}
lossWeights = {
	"class_label": 1.0,
	"bounding_box": 1.0
}
opt = Adam(lr=0.0001)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
trainTargets = {
	"class_label": trainLabels,
	"bounding_box": trainBBoxes
}
testTargets = {
	"class_label": testLabels,
	"bounding_box": testBBoxes
}

In [19]:
H = model.fit(
	trainImages, trainTargets,
	validation_data=(testImages, testTargets),
	batch_size=32,
	epochs=100,
	verbose=1)

2023-01-14 10:17:23.569415: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2800422912 exceeds 10% of free system memory.
2023-01-14 10:17:26.835223: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 2800422912 exceeds 10% of free system memory.
2023-01-14 10:17:29.147914: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2023-01-14 10:17:31.362706: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


146/146 [==============================] - ETA: 0s - loss: 1.9881 - bounding_box_loss: 0.0149 - class_label_loss: 1.9732 - bounding_box_accuracy: 0.5364 - class_label_accuracy: 0.3053

2023-01-14 10:17:52.528824: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 700256256 exceeds 10% of free system memory.
2023-01-14 10:17:53.779946: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 700256256 exceeds 10% of free system memory.


146/146 [==============================] - 28s 121ms/step - loss: 1.9881 - bounding_box_loss: 0.0149 - class_label_loss: 1.9732 - bounding_box_accuracy: 0.5364 - class_label_accuracy: 0.3053 - val_loss: 1.6195 - val_bounding_box_loss: 0.0134 - val_class_label_loss: 1.6061 - val_bounding_box_accuracy: 0.5365 - val_class_label_accuracy: 0.4807
Epoch 2/100
146/146 [==============================] - 14s 98ms/step - loss: 1.6454 - bounding_box_loss: 0.0117 - class_label_loss: 1.6337 - bounding_box_accuracy: 0.6020 - class_label_accuracy: 0.4298 - val_loss: 1.4727 - val_bounding_box_loss: 0.0116 - val_class_label_loss: 1.4611 - val_bounding_box_accuracy: 0.5933 - val_class_label_accuracy: 0.5107
Epoch 3/100
146/146 [==============================] - 14s 99ms/step - loss: 1.4847 - bounding_box_loss: 0.0107 - class_label_loss: 1.4740 - bounding_box_accuracy: 0.6381 - class_label_accuracy: 0.4874 - val_loss: 1.3946 - val_bounding_box_loss: 0.0117 - val_class_label_loss: 1.3829 - val_bounding_bo

In [20]:
model.save('/kaggle/working/detector.h5', save_format="h5")
f = open('/kaggle/working/lb.pickle', "wb")
f.write(pickle.dumps(lb))
f.close()